In [ ]:
import re
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys.electronfactors as electronfactors

In [ ]:
input_patient_id = input()
patientID = input_patient_id.zfill(6)

In [ ]:
rccc_string_search_pattern = (
    r'\\MONACODA\FocalData\RCCC\1~Clinical\*{}\plan\*\*tel.1'.format(patientID))
rccc_filepath_list = glob(rccc_string_search_pattern)

# First time accessing this need to type \\nbccc-monaco into a file browser
# and provide nbccc account credentials
nbccc_string_search_pattern = (
    r'\\nbccc-monaco\FOCALDATA\NBCCC\1~Clinical\*{}\plan\*\*tel.1'.format(patientID))
nbccc_filepath_list = glob(nbccc_string_search_pattern)


filepath_list = np.concatenate([rccc_filepath_list, nbccc_filepath_list])
filepath_list

In [ ]:
electronmodel_regex = "RiverinaAgility - (\d+)MeV"
applicator_regex = "(\d+)X\d+"

insert_data = dict()

for telfilepath in filepath_list:
    insert_data[telfilepath] = dict()
    
    with open(telfilepath, "r") as file:
        telfilecontents = np.array(file.read().splitlines())

    insert_data[telfilepath]["reference_index"] = []
    for i, item in enumerate(telfilecontents):
        if re.search(electronmodel_regex, item):
            insert_data[telfilepath]["reference_index"] += [i]

    insert_data[telfilepath]["applicators"] = [
        re.search(applicator_regex, telfilecontents[i+12]).group(1)
        for i in insert_data[telfilepath]["reference_index"]]

    insert_data[telfilepath]["energies"] = [
        re.search(electronmodel_regex, telfilecontents[i]).group(1)
        for i in insert_data[telfilepath]["reference_index"]]
    
insert_data

In [ ]:
# # Troubleshooting
# telfilepath = filepath_list[2]

# with open(telfilepath, "r") as file:
#     telfilecontents = np.array(file.read().splitlines())
    
# re.search(electronmodel_regex, telfilecontents[20])

In [ ]:
for telfilepath in filepath_list:    
    with open(telfilepath, "r") as file:
        telfilecontents = np.array(file.read().splitlines())
        
    insert_data[telfilepath]["x"] = []
    insert_data[telfilepath]["y"] = []

    for i, index in enumerate(insert_data[telfilepath]["reference_index"]):
        insert_inital_range = telfilecontents[
            index + 51::]  # coords start 51 lines after electron model name
        insert_stop = np.where(
            insert_inital_range=='0')[0][0]  # coords stop right before a line containing 0

        insert_coords_string = insert_inital_range[:insert_stop]
        insert_coords = np.fromstring(','.join(insert_coords_string), sep=',')
        insert_data[telfilepath]["x"].append(insert_coords[0::2]/10)
        insert_data[telfilepath]["y"].append(insert_coords[1::2]/10)
        
insert_data

In [ ]:
for telfilepath in filepath_list:   
    insert_data[telfilepath]["width"] = []
    insert_data[telfilepath]["length"] = []
    insert_data[telfilepath]["circle_centre"] = []
    insert_data[telfilepath]["P/A"] = []
    
    for i in range(len(insert_data[telfilepath]["reference_index"])):
    
        width, length, circle_centre = electronfactors.parameterise_insert(
            insert_data[telfilepath]["x"][i], 
            insert_data[telfilepath]["y"][i])

        insert_data[telfilepath]["width"].append(width)
        insert_data[telfilepath]["length"].append(length)
        insert_data[telfilepath]["circle_centre"].append(circle_centre)

        insert_data[telfilepath]["P/A"].append(
            electronfactors.convert2_ratio_perim_area(width, length)
        )
        
insert_data

In [ ]:
data_filename = r'./electron_factor_measured_data.csv'
data = pd.read_csv(data_filename)

In [ ]:
width_data = data['Width (cm @ 100SSD)']
length_data = data['Length (cm @ 100SSD)']
factor_data = data['RCCC Inverse factor (dose open / dose cutout)']

p_on_a_data = electronfactors.convert2_ratio_perim_area(
    width_data, length_data)

In [ ]:
for telfilepath in filepath_list:
    insert_data[telfilepath]["model_factor"] = []
    
    for i in range(len(insert_data[telfilepath]["reference_index"])):
        applicator = float(insert_data[telfilepath]["applicators"][i])
        energy = float(insert_data[telfilepath]["energies"][i])
        ssd = 100

        reference = (
            (data['Energy (MeV)'] == energy) &
            (data['Applicator (cm)'] == applicator) &
            (data['SSD (cm)'] == ssd)
        )
        
        number_of_measurements = np.sum(reference)
        
        if number_of_measurements < 8:
            insert_data[telfilepath]["model_factor"].append(np.nan)
        else:
            insert_data[telfilepath]["model_factor"].append(
                electronfactors.spline_model_with_deformability(
                    insert_data[telfilepath]["width"],
                    insert_data[telfilepath]["P/A"],
                    width_data[reference], 
                    p_on_a_data[reference],
                    factor_data[reference]
                )[0]
            )
            
insert_data

In [ ]:
def visual_circle_and_ellipse(insert_x, insert_y, width, 
                              length, circle_centre):

    t = np.linspace(0, 2 * np.pi)
    circle = {
        "x": width / 2 * np.sin(t) + circle_centre[0],
        "y": width / 2 * np.cos(t) + circle_centre[1]
    }
    
    x_shift, y_shift, rotation_angle = electronfactors.visual_alignment_of_equivalent_ellipse(
        insert_x, insert_y, width, length, None)

    rotation_matrix = np.array([
        [np.cos(rotation_angle), -np.sin(rotation_angle)],
        [np.sin(rotation_angle), np.cos(rotation_angle)]
    ])

    ellipse = np.array([
        length / 2 * np.sin(t),
        width / 2 * np.cos(t)
    ]).T

    rotated_ellipse = ellipse @ rotation_matrix

    translated_ellipse = (
        rotated_ellipse + np.array([y_shift, x_shift]))
    ellipse = {
        "x": translated_ellipse[:,1],
        "y": translated_ellipse[:,0]
    }
    
    return circle, ellipse

In [ ]:
def plot_insert(insert_x, insert_y, width, length, circle_centre):
    circle, ellipse = visual_circle_and_ellipse(
        insert_x, insert_y, width, length, circle_centre)
    
    plt.figure()
    plt.plot(insert_x, insert_y)
    plt.axis('equal')

    plt.plot(circle['x'], circle['y'])
    plt.title('Insert shape parameterisation')
    plt.xlabel('x (cm)')
    plt.ylabel('y (cm)')
    plt.grid(True)

    plt.plot(ellipse['x'], ellipse['y'])

In [ ]:
def plot_model(width_data, length_data, factor_data):

    i, j, k = electronfactors.create_transformed_mesh(
        width_data, length_data, factor_data)    
    model_width, model_length, model_factor = i, j, k

    model_width_mesh, model_length_mesh = np.meshgrid(
        model_width, model_length)
    
    vmin = np.nanmin(np.concatenate([model_factor.ravel(), factor_data.ravel()]))
    vmax = np.nanmax(np.concatenate([model_factor.ravel(), factor_data.ravel()]))
    vrange = vmax - vmin
    
    plt.scatter(
        width_data, length_data, s=100, c=factor_data,
        cmap='viridis', vmin=vmin, vmax=vmax, zorder=2
    )
    
    plt.colorbar()
    
    cs = plt.contour(
        model_width, model_length, model_factor, 20,
        vmin=vmin, vmax=vmax
    )
    
    plt.clabel(cs, cs.levels[::2], inline=True)
    
    plt.title('Insert model')
    plt.xlabel('width (cm)')
    plt.ylabel('length (cm)')

In [ ]:
for telfilepath in filepath_list:
    print("=======================================================================")
    print(telfilepath)
    
    for i in range(len(insert_data[telfilepath]["reference_index"])):
        applicator = float(insert_data[telfilepath]["applicators"][i])
        energy = float(insert_data[telfilepath]["energies"][i])
        ssd = 100
        
        width = insert_data[telfilepath]["width"][i]
        length = insert_data[telfilepath]["length"][i]
        
        print("Applicator = {} cm".format(applicator))
        print("Energy = {} MeV\n".format(energy))
        
        print("Width = {0:0.2f} cm".format(width))
        print("Length = {0:0.2f} cm".format(length))
        print("Factor = {0:0.3f}".format(
            insert_data[telfilepath]["model_factor"][i]))
        
        plt.figure()
        plot_insert(
            insert_data[telfilepath]["x"][i], 
            insert_data[telfilepath]["y"][i], 
            insert_data[telfilepath]["width"][i], 
            insert_data[telfilepath]["length"][i], 
            insert_data[telfilepath]["circle_centre"][i])
        
        reference = (
            (data['Energy (MeV)'] == energy) &
            (data['Applicator (cm)'] == applicator) &
            (data['SSD (cm)'] == ssd)
        )
        
        number_of_measurements = np.sum(reference)
        
        plt.figure()
        if number_of_measurements < 8:
            plt.scatter(
                width_data[reference], 
                length_data[reference], s=100, 
                c=factor_data[reference],
                cmap='viridis', zorder=2
            )
            plt.colorbar()        
        else:        
            plot_model(
                width_data[reference], length_data[reference], factor_data[reference])
            
            
        weighted_difference_squares = (
            (width_data[reference] - width) ** 2 + 
            0.5 * (length_data[reference] - length) ** 2
        )
        closest_cutout = np.argmin(weighted_difference_squares)
        closest_width = width_data[reference][closest_cutout]
        closest_length = length_data[reference][closest_cutout]
        closest_factor = factor_data[reference][closest_cutout]
        print('')
        print(
            "Closest cutout within data\n"
            "  Width: {0:0.2f} cm\n"
            "  Length: {1:0.2f} cm\n"
            "  Factor: {2:0.3f}".format(closest_width, closest_length, closest_factor)
        )
        
        plt.show()